In [120]:
import numpy as np
import pandas as pd

In [121]:
file_path = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/raw/mpg.csv'
df = pd.read_csv(file_path)
df.origin = df.origin.astype('str')
df.head(n=2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320


In [136]:
from bokeh.io import output_file, show, curdoc
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, Slider, Select

from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6

from bokeh.layouts import widgetbox, row

In [123]:
source = ColumnDataSource(data={'x': df[df.model_year==70].mpg, 
                                'y': df[df.model_year==70].displacement, 
                                'name': df[df.model_year==70].name, 
                                'origin': df[df.model_year==70].origin})

In [124]:
# Setting the range for the figure
xmin, xmax = min(df.mpg), max(df.mpg)
ymin, ymax = min(df.displacement), max(df.displacement)

In [125]:
# Make a color mapper: color_mapper
origin_list = df.origin.unique().tolist()
color_mapper = CategoricalColorMapper(factors=origin_list, palette=Spectral6)

In [126]:
# curdoc().add_root(p)
# curdoc().title = 'Automobile'
p = figure(title='Automobile data', plot_height=400, plot_width=700,
           tools=[HoverTool(tooltips=[('name', '@name'), ('MPG', '@x'), ('Displacement', '@y')])],
           x_range = (xmin, xmax), y_range= (ymin, ymax))


In [127]:
p.circle(x='x', y='y', source=source, color=dict(field='origin', transform=color_mapper), legend_field='origin')

GlyphRenderer(id='3484', ...)

In [128]:
p.xaxis.axis_label ='MPG'
p.yaxis.axis_label = 'Displacement'
p.legend.location = 'top_right'
p.title.text = 'Automobile data for Model Year 70'

In [129]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    year = slider.value
    x = x_select.value
    y = y_select.value
    # Label axes of plot
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y
    
    new_data = {'x': df[df.model_year==year][x], 
                'y': df[df.model_year==year][y], 
                'name': df[df.model_year==year].name, 
                'origin': df[df.model_year==year].origin }

    source.data = new_data
    
    # Set the range of all axes
    p.x_range.start = min(df[x])
    p.x_range.end = max(df[x])
    p.y_range.start = min(df[y])
    p.y_range.end = max(df[y])

    # Add title to figure: plot.title.text
    p.title.text = 'Automobile data for Model Year %d' % year


In [137]:
slider = Slider(start=70, end=82, step=1, value=70, title='Model Year')

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)

# Create a dropdown Select widget for the x data: x_select
x_select = Select(
    options=['mpg', 'horsepower', 'acceleration', 'weight', 'displacement', 'cylinders'],
    value='mpg',
    title='x-axis data'
)

# Attach the update_plot callback to the 'value' property of x_select
x_select.on_change('value', update_plot)

# Create a dropdown Select widget for the y data: y_select
y_select = Select(
    options=['mpg', 'horsepower', 'acceleration', 'weight', 'displacement', 'cylinders'],
    value='displacement',
    title='y-axis data'
)

# Attach the update_plot callback to the 'value' property of y_select
y_select.on_change('value', update_plot)

In [138]:
# Create layout and add to current document
layout = row(widgetbox(slider, x_select, y_select), p)
curdoc().add_root(layout)

RuntimeError: Models must be owned by only a single document, Selection(id='2880', ...) is already in a doc